# Cleaning RFM Online Retail Data

Import Required Libraries

In [1]:
import pandas as pd
import numpy as np
import mysql.connector
import configparser
import re

Connect to local database and read Online Retail data into a pandas dataframe, df.

In [2]:
#get connection details from a configuration file
config = configparser.ConfigParser()
config.read('nb.cfg')


try:
    #connect to database
    connection = mysql.connector.connect(host=config['mysql']['host'], database=config['mysql']['database'],\
                                         user=config['mysql']['user'], password=config['mysql']['password'])

    #read onlineretail data to a pandas dataframe
    get_data = "SELECT * FROM onlineretail;"
    df = pd.read_sql(get_data, connection)
    connection.close()

except Exception as e:
    connection.close()
    print(str(e))


Check the datatypes of each column in df.

In [3]:
print(df.dtypes)

invoiceno       object
stockcode       object
description     object
quantity         int64
invoicedate     object
unitprice      float64
customerid     float64
country         object
dtype: object


We see that quantity, unitprice, and customerid are all numerical quantities.

Find columns that contain null values.

In [4]:
print(df.isnull().any())

invoiceno      False
stockcode      False
description     True
quantity       False
invoicedate    False
unitprice      False
customerid      True
country        False
dtype: bool


Note that only description and customerid contain null values. For the purpose of my RFM analysis the null values in description are not concerning. The entries with null as customerid are removed. 

In [5]:
df = df.dropna(axis=0, subset=['customerid'])

Check for null values again.

In [6]:
print(df.isnull().any())

invoiceno      False
stockcode      False
description    False
quantity       False
invoicedate    False
unitprice      False
customerid     False
country        False
dtype: bool


Check for potentially problematic entries in quantity, invoicedate, and unitprice.

In [7]:
print('Total negative Quantities: ', len(df.loc[df['quantity'] < 0]))
print('Proportion of Negative Quantities: ' + str(len(df.loc[df['quantity'] < 0])/len(df)))
print('Total negative Unit Prices: ', len(df.loc[df['unitprice'] < 0]))
print('Proportion of Negative Unit Prices: ' + str(len(df.loc[df['unitprice'] < 0])/len(df)))

Total negative Quantities:  8905
Proportion of Negative Quantities: 0.021888803403887137
Total negative Unit Prices:  0
Proportion of Negative Unit Prices: 0.0


In [8]:
print(df.loc[df['quantity'] < 0].head())

    invoiceno stockcode                       description  quantity  \
141   C536379         D                          Discount        -1   
154   C536383    35004C   SET OF 3 COLOURED  FLYING DUCKS        -1   
235   C536391     22556    PLASTERS IN TIN CIRCUS PARADE        -12   
236   C536391     21984  PACK OF 12 PINK PAISLEY TISSUES        -24   
237   C536391     21983  PACK OF 12 BLUE PAISLEY TISSUES        -24   

         invoicedate  unitprice  customerid         country  
141   12/1/2010 9:41      27.50     14527.0  United Kingdom  
154   12/1/2010 9:49       4.65     15311.0  United Kingdom  
235  12/1/2010 10:24       1.65     17548.0  United Kingdom  
236  12/1/2010 10:24       0.29     17548.0  United Kingdom  
237  12/1/2010 10:24       0.29     17548.0  United Kingdom  


Both quantities contains negative values, but the quantities with negative values make up a small proportion of our data. Looking at the data, rows with negative quantiteies seem to be returns, and have a differently formated invoice numbers that contain the letter 'C'. For the purpose of this project I'll treat them as returns, and will not include them in the rfm analysis.
The data also includes other rows which have negative quantity. I will remove these as well.
Some enties also have 0 unit price, I'm not sure if this is a mistake, or intentional (e.g. a promotion). I'll leave these entries for now.

Remove rows with negative quantity since we do not want to include returns.

In [57]:
df = df.loc[df['quantity'] > 0]

Verify that the negative quantities have been removed. 

In [58]:
print('Total negative Quantities: ', len(df.loc[df['quantity'] < 0]))

Total negative Quantities:  0


Filter for any invoicedates that are not formatted like '(m)m/(d)d/yyyy (h)h:mm'.

In [59]:
if df.invoicedate.dtype != 'datetime64[ns]':
    #format 'mm/dd/yyyy hh:mm' as regular expression
    format = r'^([1-9]|1[0-2])/([1-9]|1[0-9]|2[0-9]|3[0,1])/(20[0-1][0-9]|202[0-2])\s([1-9]|1[0-9]|2[0-3])\:[0-5][0-9]$'
    print('Total imporperly formatted dates: ', df[[not bool(re.match(format, x)) for x in df['invoicedate']]])

Total imporperly formatted dates:  Empty DataFrame
Columns: [invoiceno, stockcode, description, quantity, invoicedate, unitprice, customerid, country]
Index: []


Convert invoicedates to datetime formats, catch an exception for invalid dates. 
ex. '2/30/2020 8:30' (February 30th) is invalid

In [60]:
try:
    df['invoicedate'] = pd.to_datetime(df['invoicedate'], format='%m/%d/%Y %H:%M')
    print("All dates in invoicedate are valid!")
except Exception as e:
    print(str(e))
    print("Invoicedate contains invalid dates!")

All dates in invoicedate are valid!


Replace np.Nan objects with None type objects, and split df into chunks to facilitate uploading into database.

In [61]:
df = df.replace({np.nan: None})

chunks = [df.iloc[5000*i:min(5000*(i + 1), len(df))] for i in range(len(df)//5000 + 1)]
del df

Connect to database. Create table to store cleaned data and replace old onlineretail table. Create table to compute and store recency, frequency, and monetary data for each customer.

In [63]:
#connect to database
try:
    connection = mysql.connector.connect(host=config['mysql']['host'], database=config['mysql']['database'],\
                                         user=config['mysql']['user'], password=config['mysql']['password'])
    curr = connection.cursor()

    #create temporary table to store df
    curr.execute("DROP TABLE IF EXISTS onlineretail_temp;")
    query = """
                CREATE TABLE onlineretail_temp (invoiceno varchar(20),
                                                stockcode varchar(20),
                                                description text,
                                                quantity int,
                                                invoicedate datetime,
                                                unitprice double(12,2),
                                                customerid int,
                                                country varchar(20));
            """
            
    curr.execute(query)

    #iterate through chunks of df 
    for chunk in chunks:

        #insert data into onlineretail_temp table
        sql = "insert into onlineretail_temp (%s) values " % (", ".join(chunk.columns)) + "(" + "%s, "*(len(chunk.columns) - 1) + "%s" + ");"
        curr.executemany(sql, chunk.values.tolist())
        connection.commit()

    #remove table onlineretail and rename onlineretail_temp as onlineretail
    curr.execute("DROP TABLE IF EXISTS onlineretail;")
    curr.execute("ALTER TABLE onlineretail_temp RENAME TO onlineretail;")
    connection.commit()

    #create a table for our consumer rfm data
    curr.execute("DROP TABLE IF EXISTS consumer_rfm;")
    create_rfm_data = """
                    CREATE TABLE consumer_rfm AS
                    SELECT customerid,
                    datediff(date_add((select max(invoicedate) from onlineretail), INTERVAL 1 DAY), max(invoicedate)) recency,
                    count(DISTINCT invoiceno) frequency,
                    sum(quantity * unitprice) monetary
                    from onlineretail
                    group by customerid; 
                """

    create_primary_key = """
                    ALTER TABLE consumer_rfm
                    ADD CONSTRAINT pk_customerid PRIMARY KEY (customerid);
                """

    curr.execute(create_rfm_data)
    curr.execute(create_primary_key)
    connection.commit()
    connection.close()
except Exception as e:
    connection.close()
    print(str(e))